In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [9]:
df = pd.read_csv(FOLDER+'train.csv')
df_structures = pd.read_csv(FOLDER+'structures.csv')

In [4]:
from process import get_dist_matrix

In [37]:
df_idx = df.set_index('molecule_name')
df_structure_idx = df_structures.set_index('molecule_name')
mols_train = df['molecule_name'].unique()

In [3]:
pred_train_name = '20190711_predict_train.pickle'
pred_val_name = '20190711_predict_val.pickle'
temp_pickle = 'features_temp_{}.pickle'
temp_pickle_val = 'features_temp_val_{}.pickle'
temp_no = 0

In [4]:
df_train = pd.read_pickle(OUTPUT + pred_train_name)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]
df_val = pd.read_pickle(OUTPUT + pred_val_name)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]

mols_train = df_train['molecule_name'].unique()
mols_val = df_val['molecule_name'].unique()

df = pd.concat([df_train, df_val], axis=0).sort_values('id')
df_idx = df.set_index('molecule_name')

In [22]:
dist_arr[focus_0, focus_atom]

IndexError: too many indices for array

In [51]:
m = mols[1000]
bond = ['1JHC', '2JHC', '3JHC']
dist_mat = get_dist_matrix(df_structure_idx, m)    

df_temp = df_idx.loc[m].query('type in {}'.format(bond))
con_id = df_temp['id'].values
df_temp_idx = df_temp.set_index('id')

features = np.zeros([len(con_id), 32]) # pred: 1x2, 1JHC: 3x2, 2JHC: 3x2, 3JHC: 9x2  

for i, idx in enumerate(con_id):
    pred_cols = 'scalar_coupling_constant'
    features[i, 0] = df_temp_idx.loc[idx][pred_cols]

    focus_atom = df_temp_idx.loc[idx]['atom_index_1']
    dist_arr = dist_mat[focus_atom]
    focus_0 = df_temp_idx.loc[idx]['atom_index_0']

    features[i, 1] = 1/dist_arr[focus_0]
    
    df_temp_idx_1JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom)) 
    atoms_1JHC = df_temp_idx_1JHC['atom_index_0']
    predicts_1JHC = df_temp_idx_1JHC[pred_cols]
    dist_1JHC = dist_arr[atoms_1JHC]
    sorting_1JHC = np.argsort(dist_1JHC)
    features[i, 2:2+len(predicts_1JHC)] = predicts_1JHC.values[sorting_1JHC]    
    features[i, 5:5+len(predicts_1JHC)] = 1/dist_arr[atoms_1JHC][sorting_1JHC]

    df_temp_idx_2JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[1], focus_atom)) 
    atoms_2JHC = df_temp_idx_2JHC['atom_index_0']
    predicts_2JHC = df_temp_idx_2JHC[pred_cols]
    dist_2JHC = dist_arr[atoms_2JHC]
    sorting_2JHC = np.argsort(dist_2JHC)
    features[i, 8:8+len(predicts_2JHC)] = predicts_2JHC.values[sorting_2JHC]   
    features[i, 11:11++len(predicts_2JHC)] = 1/dist_arr[atoms_2JHC][sorting_2JHC]
    
    df_temp_idx_3JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[2], focus_atom)) 
    atoms_3JHC = df_temp_idx_3JHC['atom_index_0']
    predicts_3JHC = df_temp_idx_3JHC[pred_cols]
    dist_3JHC = dist_arr[atoms_3JHC]
    sorting_3JHC = np.argsort(dist_3JHC)    
    features[i, 14:14+len(predicts_3JHC)] = predicts_3JHC.values[sorting_3JHC]
    features[i, 23:23+len(predicts_3JHC)] = 1/dist_arr[atoms_3JHC][sorting_3JHC]

In [50]:
predicts_1JHC.values[sorting_1JHC]

array([85.9039, 87.2567])

In [52]:
features[0]

array([81.8527    ,  0.91543847, 82.3464    , 83.5175    ,  0.        ,
        0.91331381,  0.91159914,  0.        , -1.48222   ,  0.        ,
        0.        ,  0.46646755,  0.        ,  0.        ,  3.58046   ,
        2.15835   ,  1.88922   ,  9.75857   ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.37368453,  0.35522501,
        0.35405915,  0.2874866 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [24]:
df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom))[pred_cols]

id
113    83.5143
119    80.7047
Name: scalar_coupling_constant, dtype: float64

In [41]:
dist_arr[atoms_1JHC]

array([], dtype=float64)

In [53]:
bond = ['1JHC', '2JHC', '3JHC']

div = len(mols_train) // 8

df_train_features = pd.DataFrame()
start0 = time.time()

# for m in mols_train[div*temp_no:div*(temp_no+1)]:
for m in mols_train[:10]:
    
    if type(df_idx.loc[m]) == pd.Series:
        continue
    bond = ['1JHC', '2JHC', '3JHC']
    dist_mat = get_dist_matrix(df_structure_idx, m)    

    df_temp = df_idx.loc[m].query('type in {}'.format(bond))
    con_id = df_temp['id'].values
    df_temp_idx = df_temp.set_index('id')

    features = np.zeros([len(con_id), 32]) # pred: 1x2, 1JHC: 3x2, 2JHC: 3x2, 3JHC: 9x2  

    for i, idx in enumerate(con_id):
        pred_cols = 'scalar_coupling_constant'
        features[i, 0] = df_temp_idx.loc[idx][pred_cols]

        focus_atom = df_temp_idx.loc[idx]['atom_index_1']
        dist_arr = dist_mat[focus_atom]
        focus_0 = df_temp_idx.loc[idx]['atom_index_0']

        features[i, 1] = 1/dist_arr[focus_0]

        df_temp_idx_1JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom)) 
        atoms_1JHC = df_temp_idx_1JHC['atom_index_0']
        predicts_1JHC = df_temp_idx_1JHC[pred_cols]
        dist_1JHC = dist_arr[atoms_1JHC]
        sorting_1JHC = np.argsort(dist_1JHC)
        features[i, 2:2+len(predicts_1JHC)] = predicts_1JHC.values[sorting_1JHC]    
        features[i, 5:5+len(predicts_1JHC)] = 1/dist_arr[atoms_1JHC][sorting_1JHC]

        df_temp_idx_2JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[1], focus_atom)) 
        atoms_2JHC = df_temp_idx_2JHC['atom_index_0']
        predicts_2JHC = df_temp_idx_2JHC[pred_cols]
        dist_2JHC = dist_arr[atoms_2JHC]
        sorting_2JHC = np.argsort(dist_2JHC)
        features[i, 8:8+len(predicts_2JHC)] = predicts_2JHC.values[sorting_2JHC]   
        features[i, 11:11++len(predicts_2JHC)] = 1/dist_arr[atoms_2JHC][sorting_2JHC]

        df_temp_idx_3JHC = df_temp_idx.loc[con_id != idx].query('type == "{}" and atom_index_1 == {}'.format(bond[2], focus_atom)) 
        atoms_3JHC = df_temp_idx_3JHC['atom_index_0']
        predicts_3JHC = df_temp_idx_3JHC[pred_cols]
        dist_3JHC = dist_arr[atoms_3JHC]
        sorting_3JHC = np.argsort(dist_3JHC)    
        features[i, 14:14+len(predicts_3JHC)] = predicts_3JHC.values[sorting_3JHC]
        features[i, 23:23+len(predicts_3JHC)] = 1/dist_arr[atoms_3JHC][sorting_3JHC]

    df_feature = pd.DataFrame(features)
    df_feature['id'] = con_id
    df_train_features = pd.concat([df_train_features, df_feature], axis=0)
    
# df_train_features.to_pickle(OUTPUT + temp_pickle.format(temp_no))
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

total elapsed_time:1.06654953956604[sec]


In [54]:
df_train_features

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,id
0,84.80760,0.915790,84.8093,84.8095,84.8074,0.915796,0.915795,0.915791,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
1,84.80740,0.915791,84.8093,84.8095,84.8076,0.915796,0.915795,0.915790,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
2,84.80930,0.915796,84.8095,84.8074,84.8076,0.915795,0.915791,0.915790,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,7
3,84.80950,0.915795,84.8093,84.8074,84.8076,0.915796,0.915791,0.915790,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
0,171.22000,0.937560,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,17
0,83.54290,0.913281,83.5417,83.5484,0.0000,0.913277,0.913269,0.000000,-2.37852,-2.37876,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
1,-2.37831,0.458192,83.5430,83.5418,83.5486,0.913281,0.913277,0.913269,-2.37862,-2.37716,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,20
2,83.54170,0.913277,83.5429,83.5484,0.0000,0.913281,0.913269,0.000000,-2.37852,-2.37876,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,26
3,-2.37862,0.458193,83.5430,83.5418,83.5486,0.913281,0.913277,0.913269,-2.37831,-2.37716,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,27
4,83.54840,0.913269,83.5429,83.5417,0.0000,0.913281,0.913277,0.000000,-2.37852,-2.37876,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32


In [ ]:
bond = ['1JHC', '2JHC', '3JHC']
temp_pickle_val = 'features_temp_val_{}.pickle'
temp_no = 0
div = len(mols_val) // 8

df_val_features = pd.DataFrame()
start0 = time.time()

for m in mols_val[div*temp_no:div*(temp_no+1)]:
    if type(df_idx.loc[m]) == pd.Series:
        continue
    df_temp = df_idx.loc[m].query('type in {}'.format(bond))
    con_id = df_temp['id'].values
    df_temp_idx = df_temp.set_index('id')

    featured_sc = np.zeros([0,4])
    for idx in con_id:
        pred_cols = 'predict'
        
        pred = df_temp_idx.loc[idx][pred_cols]
        
        focus_atom = df_temp_idx.loc[idx]['atom_index_1']

        mean_1JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom))\
                        [pred_cols].mean()

        mean_2JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[1], focus_atom))\
                        [pred_cols].mean()

        mean_3JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[2], focus_atom))\
                        [pred_cols].mean()

        means = np.array([pred, mean_1JHC, mean_2JHC, mean_3JHC]).reshape(-1,4)
        featured_sc = np.vstack([featured_sc, means])

    df_feature = pd.DataFrame(featured_sc)
    df_feature['id'] = con_id
    df_val_features = pd.concat([df_val_features, df_feature], axis=0)
    
df_val_features.to_pickle(OUTPUT + temp_pickle_val.format(temp_no))
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
# df_train_features_list = []
# for n in range(8):
# df_train_features_list.append(pd.read_pickle(OUTPUT + temp_pickle.format(n)))

# df_train_features = pd.concat(df_train_features_list, axis=0)

In [ ]:
# def type_score(y_val, y_pred):
#     return np.log(sum(np.abs(y_val- y_pred)) / len(y_val))